In [2]:
# import libraries
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# model parameters 
min_x = 0.
max_x = 200000.
min_y = 0.
max_y = 200000.
min_z = -200000.
max_z = 0.
print("min_x: {:.1f} m, max_x: {:.1f} m".format(min_x,max_x))
print("min_y: {:.1f} m, max_y: {:.1f} m".format(min_y,max_y))
print("min_z: {:.1f} m, max_z: {:.1f} m".format(min_z,max_z))

min_x: 0.0 m, max_x: 200000.0 m
min_y: 0.0 m, max_y: 200000.0 m
min_z: -200000.0 m, max_z: 0.0 m


In [4]:
# calculate model's number of elements
fmax = 0.5 # maximum frequency in Hz
elements_per_wavelength = 3. # resolution criterion
min_vel = 3000.
hmax =  (min_vel / fmax / elements_per_wavelength) # element size
nelem_x = int(np.abs(max_x) / hmax) # number of elements in x domain
nelem_y = int(np.abs(max_y) / hmax) # number of elements in y domain
nelem_z = int(np.abs(min_z) / hmax) # number of elements in z domain

print("hmax: {} m".format(hmax))
print("nelem_x: {}".format(nelem_x))
print("nelem_y: {}".format(nelem_y))
print("nelem_z: {}".format(nelem_z))

hmax: 2000.0 m
nelem_x: 100
nelem_y: 100
nelem_z: 100


In [5]:
# build a structured grid
from pymesher.structured_grid_3D import StructuredGrid3D

sg = StructuredGrid3D.cube(nelem_x=nelem_x,
                           nelem_y=nelem_y,
                           nelem_z=nelem_z,
                           min_x=min_x,
                           max_x=max_x,
                           min_y=min_y,
                           max_y=max_y,
                           min_z=min_z,
                           max_z=max_z)

# convert the structured grid to an unstructured mesh
m = sg.get_unstructured_mesh()

In [6]:
# from PREM earth model profile at depth 6368-6356 km (homogenenous isotropic model)
vp  = 5800. # 5800 m/s
vs  = 3200. # 3200 m/s
rho = 2600. # 2600 g/cc

# attach our model to the actual mesh    
m.attach_field("VP",  vp  * np.ones(m.npoint))
m.attach_field("VS",  vs  * np.ones(m.npoint))
m.attach_field("RHO", rho * np.ones(m.npoint))
m.attach_field("fluid",     np.zeros(m.nelem))

# find outer boundaries of the domain assuming the domain is rectangular
m.find_side_sets(mode='cartesian')

# write model in exodus format that can be read by paraview and salvus
m.write_exodus("/import/deadlock-data/taufiqurrahman/MESS2018/experiment_Maeda2016/mesh_3d_homogeneous.e")

In [14]:
# create toml file which contains seismic source position at certain depth 
with open("/import/deadlock-data/taufiqurrahman/MESS2018/experiment_Maeda2016/source.toml", "w") as text_file:
    name = "0000"
    srcLocX = 127539. #125000.
    srcLocY = 76823.  #75000.
    srcLocZ = -60000.
    spatial_type = "moment_tensor"
    temporal_type = "ricker"
    scaleX = 1e9
    scaleY = 1e9
    scaleZ = 1e9
    center_frequency = fmax/2 
    print("\n[[source]]", file=text_file)
    print("name = \"{0}\"".format(name), file=text_file)
    print("location = [{0}, {1}, {2}]".format(srcLocX, srcLocY, srcLocZ), file=text_file)
    print("spatial_type = \"{0}\"".format(spatial_type), file=text_file)
    print("temporal_type = \"{0}\"".format(temporal_type), file=text_file)
    print("center_frequency = {0}".format(center_frequency), file=text_file)
    print("scale = [{0}, {1}, {2}, 0.0, 0.0, 0.0]".format(scaleX, scaleY, scaleZ), file=text_file)

In [15]:
! cat /import/deadlock-data/taufiqurrahman/MESS2018/experiment_Maeda2016/source.toml


[[source]]
name = "0000"
location = [127539.0, 76823.0, -60000.0]
spatial_type = "moment_tensor"
temporal_type = "ricker"
center_frequency = 0.25
scale = [1000000000.0, 1000000000.0, 1000000000.0, 0.0, 0.0, 0.0]
